In [12]:
import tweepy
from pprint import pprint
import pandas as pd

pd.options.display.max_rows = 100

In [13]:

#Creacion SparkSession

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('twitter_structured_streaming').getOrCreate()
# Librerias 

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [14]:
#define schema para el input data
schema=StructType().add("user_id", "string")\
.add("user_name", "string").add("text", "string")\
.add("location", "string").add("user_location","string")\
.add("user_followers_count","integer").add("user_created_at",'string')\
.add("user_count_tweets","integer").add("hashtag",'string')
schema

StructType(List(StructField(user_id,StringType,true),StructField(user_name,StringType,true),StructField(text,StringType,true),StructField(location,StringType,true),StructField(user_location,StringType,true),StructField(user_followers_count,IntegerType,true),StructField(user_created_at,StringType,true),StructField(user_count_tweets,IntegerType,true),StructField(hashtag,StringType,true)))

In [19]:
TWITTER_APP_KEY = 'RNbWRckUdr1mzMYe2ir2S44aP'
TWITTER_APP_SECRET = 'FVB0RX0nGo1VMRAMF603g7jOKuZdKfCLo0SjXWv2T15wHvLMRy'
TWITTER_KEY = '1220749681900191744-JDImKJjEF4NadKsiKekx82zDI9FyQW'
TWITTER_SECRET = 'bQaUWESSjNkqhO3d2Sc4pLPmNDYLr5DiRIIS5ko5Sg4yv'

auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY, TWITTER_SECRET)

api = tweepy.API(auth)

class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
#       pprint(status._json)
        print(status.entities['hashtags'])
        print(status.text)
        
        #spark.createDataFrame([(status.user.id_str, status.user.name, status.text, status.geo,status.user.location,status.user.followers_count,status.user.created_at,status.user.statuses_count)]
                              #,["user_id", "user_name" ,"text","location","user_location","user_followers_count","user_created_at","user_count_tweets"],schema=schema).write.csv("twitter_data",mode='append')  
        
        
#         data = (status.user.id_str, status.user.name, status.text, 
#                                 status.geo,status.user.location,status.user.followers_count,
#                                 status.user.created_at,status.user.statuses_count,[text['text'] for text in status.entities['hashtags']])
        for text in status.entities['hashtags']:
            data = (status.user.id_str, status.user.name, status.text, 
                                status.geo,status.user.location,status.user.followers_count,
                                status.user.created_at,status.user.statuses_count,text['text'])
            spark.createDataFrame([(data)],schema=schema).write.csv("twitter_data",mode='append')
        
    def on_error(self, status_code):
        if status_code == 420:
            return False

stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)


In [20]:
stream.filter(track=["#trump"])
# stream.filter()

[]
RT @AndyOstroy: This speech might just end up being the longest, most pathetic MAGAsturbatory cry-for-love kidnapping by the bruised ID of…
[]
I did nothing wrong apart from blackmail foreign leaders, pay off porn stars, assault women, abandon the kurds, ign… https://t.co/PPZEtGfA6a
[]
Having Wm. Barr in charge of the DOJ is as bad as having ⁦@LindseyGrahamSC⁩ chairing the Senate Judiciary Committee… https://t.co/BdTpPPFgPk
[{'text': 'trump', 'indices': [17, 23]}, {'text': 'Impeachment', 'indices': [24, 36]}, {'text': 'Way', 'indices': [37, 41]}]
@WatchYourRepsSC #trump #Impeachment #Way has a nice ring to it.


KeyboardInterrupt: 

In [60]:
data=spark.readStream.option("sep", ",").schema(schema).csv("twitter_data")

In [12]:
data.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- text: string (nullable = true)
 |-- location: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_followers_count: integer (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_count_tweets: integer (nullable = true)
 |-- hashtag: string (nullable = true)



In [8]:
count = data.groupBy("user_name").count()
query = (count.writeStream.queryName('count_query').outputMode('complete').format('memory').start())
# spark.sql("select * from count_query").toPandas().head()


In [68]:
spark.sql("select * from count_query").toPandas()

,user_name,count
0,Rod,1
1,Puck Arks ⏳,4
2,𝙼𝚒𝚔𝚎 𝙱𝚛𝚊𝚗𝚍,1
3,Hermione,1
4,Chevalier Noir,1
5,Mark Bahrenfuss,1
6,Jacqueline Reisinger,3
7,Gina Eaton,3
8,Yoda 14120,1
9,Sukie...🌹IM🍑ED!,1


In [86]:
# data.show().writeStream.queryName('count_query').outputMode('complete').format('memory').start()
d = data.filter(data['user_name']!='None')
d = d.groupBy(["hashtag","user_name"]).count()
# d = data.describe(['user_followers_count']).show()
query_2 = d.writeStream.queryName('pruebaa').outputMode('complete').format('memory').start()
# d= data.writeStream.format('console').start()

In [94]:
df= spark.sql("select * from pruebaa").toPandas()

In [95]:
df

,hashtag,user_name,count
0,Trump,Trading MarcoDaCosta,1
1,DNC,Puck Arks ⏳,1
2,trump,Rod,1
3,mentalhealth,Jacqueline Reisinger,1
4,Cult45,Jacqueline Reisinger,1
5,Trump,Ryk Gye,1
6,None,Mark Bahrenfuss,1
7,medaloffreedom,Suec 🇺🇸🌎,1
8,"[UNSC, Trump]",Daily News Egypt,1
9,Trump,Alana Austin,1


In [85]:
query_2.stop()